# Testing Double Descent

References:

Belkin, et al. (PNAS, 2019)
"Reconciling modern machine-learning practice and the classical bias–variance trade-off"

In [ ]:
import torch
import torch.optim as opt
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from torchvision import datasets, transforms

import matplotlib.pyplot as plt

In [ ]:
# set layer_list to be a list of the number of nodes in a given layer

class Net(nn.Module):

    def __init__(self, layer_list):
        super().__init__()
        prev = 28 * 28 # MNIST dimension
        self.llen = len(layer_list)
        self.fc = [0 for i in range(self.llen)]
        for i in range(self.llen):
            fc_num = layer_list[i]
            exec(f"self.fc{i} = nn.Linear(prev, fc_num)")
            exec(f"self.fc[i] = self.fc{i}")
            prev  = fc_num
    
        # ReLU
        self.relu = nn.ReLU()

        # last layer    
        self.final_layer = nn.Linear(prev, 10)
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        x = x.view((x.shape[0], -1))

        x = self.fc0(x)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu(x)

        # for i in range(self.llen):
        #     x = self.fc[i](x) # Linear
        #     x = self.relu(x) # ReLU
        
        x = self.final_layer(x)
        output = self.log_softmax(x)
        
        return output

In [ ]:
# testing nn

layer_list = [1000,1000,1000]
model = Net(layer_list)

input_ = torch.rand(1,28,28)
out = model(input_)

print(f"size: {out.shape}")
print(f"max arg: {torch.argmax(out)}")


## Train Neural Network

In [ ]:
# Set layers
layer_list = [1000,1000,1000,1000]
model = Net(layer_list)

In [ ]:
# Import datasets

# training set
dataset_mnist_train = datasets.MNIST('./data', 
    train=True, 
    download=True, 
    transform=transforms.Compose([transforms.ToTensor(), 
        transforms.Normalize((0.1307,), (0.3081,))]) # MNIST parameters
) 

# test set
dataset_mnist_test = datasets.MNIST('./data', 
    train=False, 
    download=True, 
    transform=transforms.Compose([transforms.ToTensor(), 
        transforms.Normalize((0.1307,), (0.3081,))]) # MNIST parameters
) 


train_loader = data_utils.DataLoader(dataset_mnist_train,
    batch_size=128,
    shuffle=True)

test_loader = data_utils.DataLoader(dataset_mnist_test,
    batch_size=128,
    shuffle=False)

In [ ]:
parameters_to_optimize = model.parameters()

# stochastic gradient descent
optimizer = opt.SGD(model.parameters(), lr=0.1, momentum=0.9)

# log-likelihood loss
loss_fn = nn.NLLLoss()

In [ ]:
def train(model, loss_fn, device, train_loader, test_loader, optimizer):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # print(torch.argmax(output, 1))
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f'[{batch_idx * len(data)}/{len(train_loader.dataset)}]\tLoss: {loss.item()}')
            test(model, loss_fn, device, test_loader)

def test(model, loss_fn, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    print(f'\nTest set Accuracy: {correct}/{len(test_loader.dataset)} ({100.*correct/len(test_loader.dataset):.1f}%)\n')


In [ ]:
train(model, loss_fn, 'cpu', train_loader, test_loader, optimizer)

In [ ]:
test(model, loss_fn, "cpu", test_loader)

## Change parameters to see Double Descent

 Number of training data parameters: 
 $60{,}000 \times 28 \times 28 = 47{,}040{,}000$

 -> We should see peak around that point
  $5\times10^{7}\simeq 3000*3000*6$ 

  (Exact number for $6$ hidden layers is $893.8$ per layer)

In [ ]:
# refine functions for multiple training

def train(model, loss_fn, device, train_loader, test_loader, optimizer):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # print(torch.argmax(output, 1))
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

def test(model, loss_fn, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    print(f'Test set Accuracy: {correct}/{len(test_loader.dataset)} ({100.*correct/len(test_loader.dataset):.1f}%)')

    # Error rate -> use to draw graph
    return 1 - correct/len(test_loader.dataset)

In [ ]:
# initialize
layer_list = np.array([25,25,25,25,25,25])
results = []

for i in range(500):
    layer_list = np.array(layer_list + 10, dtype=int) 
    model = Net(layer_list)
    parameters_to_optimize = model.parameters()
    optimizer = opt.SGD(model.parameters(), lr=0.1, momentum=0.9)

    train(model, loss_fn,'cpu',train_loader,test_loader,optimizer)
    results.append(test(model, loss_fn, "cpu", test_loader))

In [ ]:
layer_list = []
init = 25
for i in range(500):
    init += 10 
    layer_list.append(init)

In [ ]:
import scipy
from scipy.optimize import curve_fit

def expo(x, a, b):
    return a * scipy.exp(-x) +b

In [ ]:
layer_list11 = layer_list1[:23]
results11 = results1[:23]

hidden_n = layer_list11 + layer_list2 + layer_list3 + layer_list4 + layer_list5 + layer_list6 + layer_list7 + layer_list8
error_rate = results11 + results2 + results3 + results4 + results5 + results6 + results7 + results8

hidden_n, error_rate = zip(*sorted(zip(hidden_n, error_rate)))

In [ ]:
param, _ = curve_fit(expo, layer_list7, results7)

In [ ]:
fit_x = np.linspace(300,5000,100)
fit_y = expo(fit_x, *param)

In [ ]:
def param_n(hidden):
    return 5 * hidden ** 2 + 794 * hidden

In [ ]:
plt.plot(param_n(np.array(hidden_n)), error_rate)
# plt.plot(fit_x, fit_y)
# xlim(100, 120000000)
plt.show()

In [ ]:
import pickle

file_name1 = "layers.txt"
file_name2 = "results.txt"

open_file = open(file_name1, "wb")
pickle.dump(hidden_n, open_file)
open_file.close()

open_file = open(file_name2, "wb")
pickle.dump(error_rate, open_file)
open_file.close()
